<a href="https://colab.research.google.com/github/liuxiao916/Dive_into_DL_PyTorch/blob/main/5_1_%E4%BA%8C%E7%BB%B4%E5%8D%B7%E7%A7%AF%E5%B1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 5.1 二维卷积层 

### 5.1.1 二维互相关运算

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Dive Into DL
import d2lzh_pytorch

/content/drive/MyDrive/Dive Into DL


In [3]:
import torch 
from torch import nn

In [4]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return Y

In [5]:
X = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
K = torch.tensor([[0,1],[2,3]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

### 5.1.2 二维卷积层

In [6]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()
        

In [7]:
a = nn.Parameter(torch.randn(1))

### 5.1.3 图像中物体边缘检测

In [8]:
X = torch.ones((6,8))
X[:,2:6] = 0 
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
K = torch.tensor([[1,-1]])

In [10]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

### 5.1.4 通过数据学习核数组

In [11]:
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

conv2d = nn.Conv2d(1,1,(1,2))
lr = 0.01

for i in range(20):
    y_hat = conv2d(X)
    l = ((y_hat - Y)**2).sum()
    l.backward()

    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad

    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if (i+1)%2 == 0:
        print('batch %d, loss %.3f' % (i+1,l.sum().item()))

batch 2, loss 6.252
batch 4, loss 3.225
batch 6, loss 1.859
batch 8, loss 1.084
batch 10, loss 0.638
batch 12, loss 0.377
batch 14, loss 0.224
batch 16, loss 0.133
batch 18, loss 0.080
batch 20, loss 0.048


In [12]:
conv2d.weight.data

tensor([[[[ 0.9475, -0.9421]]]])